In [227]:
import geopandas as gp
import pandas as pd
import folium as fol
import matplotlib.pyplot as plt
import folium.plugins as plug
from folium.plugins import MarkerCluster
from shapely.geometry import Point, LineString, MultiLineString
from shapely import Geometry
import numpy as np
import pyproj as prj

#### Preprocessing:

##### MBTA Arc Shape File:

In [228]:
# Load in ARC Shape file
arc_gdf = gp.read_file('geodata/MBTA_ARC.shp')

# Remove Silver Line
arc_gdf.drop(arc_gdf[arc_gdf.LINE == 'SILVER'].index, inplace=True)
arc_gdf

,LINE,ROUTE,GRADE,SHAPE_LEN,geometry
0,ORANGE,Forest Hills to Oak Grove,1,1342.326419,"LINESTRING (234806.058 905421.773, 234805.768 ..."
1,ORANGE,Forest Hills to Oak Grove,4,368.215569,"LINESTRING (234806.833 905053.560, 234806.833 ..."
2,ORANGE,Forest Hills to Oak Grove,1,3010.706909,"LINESTRING (235735.315 902429.206, 235694.225 ..."
3,GREEN,C - Cleveland Circle,2,3803.501983,"LINESTRING (228902.422 898464.344, 229190.733 ..."
4,GREEN,D E,4,1621.644873,"LINESTRING (235839.891 901789.977, 235797.490 ..."
...,...,...,...,...,...
119,RED,B - Braintree C - Alewife,4,356.328195,"LINESTRING (236938.312 896104.302, 236939.741 ..."
137,GREEN,D - Union Square,4,906.641859,"LINESTRING (234518.634 902706.022, 234485.866 ..."
138,GREEN,E - Medford/Tufts,1,4710.683462,"MULTILINESTRING ((234241.720 903021.703, 23424..."
139,GREEN,E - Medford/Tufts,4,422.992560,"LINESTRING (234518.634 902706.022, 234518.353 ..."


##### MBTA Node Shape File:

In [229]:
# Load in Node Shape file
node_gdf = gp.read_file('geodata/MBTA_NODE.shp')
node_gdf.drop(node_gdf[node_gdf.LINE == 'SILVER'].index, inplace=True)
node_gdf['STATION_lower'] = node_gdf.STATION.apply(str.lower)

stop_names = set(node_gdf.STATION.array)
node_gdf

{'Museum Of Fine Arts', 'Brookline Village', 'North Quincy', 'Butler', 'Newton Highlands', 'Park Street', 'Haymarket', 'Arlington', 'Savin Hill', 'Symphony', 'Central Avenue', 'Ruggles', 'Chestnut Hill', 'Valley Road', 'Massachusetts Ave', 'Malden Center', 'South Station', 'South Street', 'Back Bay', 'Griggs Street', 'Broadway', 'Tappan Street', 'Wood Island', 'Chinatown', 'Boston University Central', 'Porter', 'Medford/Tufts', 'Science Park/West End', 'Kendall/MIT', 'Beaconsfield', 'Wellington', 'Ashmont', 'Hynes Convention Center', 'Beachmont', 'Milton', 'Riverside', 'Jackson Square', 'Warren Street', 'JFK/UMass', 'Andrew', 'Fenwood Road', 'Allston Street', 'Oak Grove', 'Copley', 'Forest Hills', 'North Station', 'Boston University East', 'Brigham Circle', 'Dean Road', 'Sullivan Square', 'Stony Brook', 'Chestnut Hill Avenue', 'Washington Square', 'Cedar Grove', 'Union Square', 'Green Street', 'Brandon Hall', 'Harvard', 'Newton Centre', 'Northeastern', 'Quincy Center', 'Saint Marys Str

,STATION,LINE,TERMINUS,ROUTE,geometry,STATION_lower
0,Park Street,GREEN/RED,N,GREEN B C D E / RED A - Ashmont B - Braintree...,POINT (236064.005 900737.761),park street
1,JFK/UMass,RED,N,A - Ashmont B - Braintree C - Alewife,POINT (236899.089 896780.048),jfk/umass
2,State,BLUE/ORANGE,N,BLUE Bowdoin to Wonderland / ORANGE Forest Hil...,POINT (236427.189 901016.630),state
3,Roxbury Crossing,ORANGE,N,Forest Hills to Oak Grove,POINT (233318.703 897936.590),roxbury crossing
42,Hynes Convention Center,GREEN,N,B C D,POINT (234003.161 899802.946),hynes convention center
...,...,...,...,...,...,...
165,Newton Highlands,GREEN,N,D - Riverside,POINT (224275.009 896915.911),newton highlands
166,Eliot,GREEN,N,D - Riverside,POINT (223362.442 896537.835),eliot
167,Woodland,GREEN,N,D - Riverside,POINT (221158.522 898069.784),woodland
168,Riverside,GREEN,Y,D - Riverside,POINT (220391.713 898574.647),riverside


In [230]:
# Load prj file to convert MBTA CRS to Lattitude and Longitude
input_prj = prj.Proj(open('geodata/MBTA_NODE.prj').read())
transformer = prj.Transformer.from_proj(input_prj, input_prj.to_latlong())

def to_lat_long(x_in, y_in):
    ''' 
    Convert the X and Y coordinates in MBTA CRS to Lattitude and Longitude
    '''
    return transformer.transform(x_in, y_in)

In [231]:
# Load the stops information to convert between GTFS Stop ID and the Stop Name
stops_df = pd.read_csv('mbta_gtfs/stops.txt')

In [232]:
def find_line_from_points(point1, point2):
    ''' 
    Given two points, find the line which is closest to both of them using the Arc shapes.
    Return the geometry defining the line closest to both of the points.
    '''
    lines_dist_to_pts = arc_gdf.geometry.apply(lambda line: line.distance(point1) + line.distance(point2))
    index = np.argmin(lines_dist_to_pts)
    return arc_gdf.iloc[index]['geometry']

def get_line_location_from_stop_id(stop_id:str):
    ''' 
    Given a stop_id defining a line (two stop_ids with a '|' separating them), return the midpoint location along
    the Arc line between them.
    '''
    station_1_id, station_2_id = stop_id.split(' | ')
    station_1_point = get_location_from_stop_id(station_1_id)
    station_2_point = get_location_from_stop_id(station_2_id)

    line = find_line_from_points(station_1_point, station_2_point)

    # Returns the distance from along the line of the given points
    station_1_dist = line.line_locate_point(station_1_point)
    station_2_dist = line.line_locate_point(station_2_point)
    # Sort them so the lower one is first so that the midpoint is between them
    distances_list = [station_1_dist, station_2_dist]
    distances_list.sort()

    # Choose random point along the line.
    # This option is less optimal for clustering.
    # random_dist = np.random.randint(distances_list[0], distances_list[1])

    # Choose midpoint of between stations
    diff = distances_list[0] - distances_list[1]
    dist = distances_list[0] + diff

    return line.line_interpolate_point(dist)

def get_location_from_stop_id(stop_id):
    ''' 
    Given the stop id defining the line (two stop_ids separated by '|') or a single stop id
    defining a station, get the geometry which defines the location.
    For a stop_id defining a line, returns the midpoint along the line.
    For a stop_id defining a station, returns the node geometry of the station.
    '''
    if '|' in stop_id:
        return get_line_location_from_stop_id(stop_id)
    
    name = stop_name_from_stop_id(stop_id)
    if name.lower() not in set(node_gdf['STATION_lower'].array):
        if "'" in name: name = name.replace("'", '')
        elif 'Ave' in name: name = name[: -3]
        elif name == 'Northeastern University': name = 'northeastern'
    geometry = node_gdf.geometry[node_gdf['STATION_lower'] == name.lower()].values[0]
    return geometry

def stop_name_from_stop_id(stop_id):
    ''' 
    For a given stop id, return the name of the station using the stops.txt file.
    '''
    return stops_df.stop_name[stops_df['stop_id'] == stop_id].values[0]

In [233]:
# Create the base map with the arc data and tooltips and popups for the information. Color by line.
base_map = arc_gdf.explore(
    color=arc_gdf['LINE'],
    tooltip='LINE',
    popup=['LINE', 'ROUTE']
)

# Add the node geometry to the base map, and define tooltips and popups for the station information.
node_gdf.explore(
    m=base_map,
    color='black',
    tooltip='STATION',
    popup=['STATION', 'LINE', 'ROUTE']
)

In [234]:
def format_speed(str):
    ''' 
    Format speed strings so that they always have a leading zero in front if they are single digits.
    This is necessary for filtering on the Folium map.
    '''
    speed = int(str[:-3])
    return f'{speed:02} mph'

In [235]:
# Read the speed restriction data 
sr_df = pd.read_csv('speed_restrictions.csv')
# Format the Restriction speed for filtering
sr_df.Restriction_Speed_MPH = sr_df.Restriction_Speed_MPH.apply(format_speed)
sr_df

,Unnamed: 0,ID,Loc_GTFS_Stop_ID,Restriction_Status,Restriction_Reason,Track_Direction,Line,Branch,Track_Name,Location_Type,Restriction_Speed_MPH,Restriction_Distance_Feet,Line_Restricted_Track_Pct,Systemwide_Restricted_Track_Pct,SR_Restriction_Distance_Span,Restriction_Path,start_date,end_date,Restriction_Length_Days
0,0,20,place-aport,Active Restriction,NaN,WB,Blue Line,Blue Line,BL WB,Station,10 mph,101.0,0.001533,0.000140,Multi-Segment,End,2023-10-17,2023-10-19,2 days
1,1,20,place-wimnl | place-aport,Active Restriction,NaN,WB,Blue Line,Blue Line,BL WB,Between Stations,10 mph,699.0,0.010608,0.000970,Multi-Segment,Start,2023-10-17,2023-10-19,2 days
2,2,21,place-mvbcl | place-aport,Active Restriction,Track,EB,Blue Line,Blue Line,BL EB,Between Stations,10 mph,400.0,0.006070,0.000555,Single Segment,Start|End,2023-12-14,2023-12-31,17 days
3,3,43,place-astao | place-welln,Active Restriction,Track,NB,Orange Line,Orange Line,OL NB,Between Stations,10 mph,1000.0,0.008403,0.001388,Single Segment,Start|End,2023-10-03,2023-10-12,9 days
4,4,44,place-rugg,Active Restriction,Track,NB,Orange Line,Orange Line,OL NB,Station,25 mph,400.0,0.003361,0.000555,Multi-Segment,Start,2023-10-17,2023-12-21,65 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,724,577753,place-mdftf | place-balsq,Active Restriction,Track,WB,Green Line,Green Line Trunk,GL Kenmore-College Ave WB,Between Stations,03 mph,400.0,0.001403,0.000555,Single Segment,Start|End,2023-09-20,2023-10-10,20 days
725,725,577754,place-balsq | place-mgngl,Active Restriction,Track,WB,Green Line,Green Line Trunk,GL Kenmore-College Ave WB,Between Stations,03 mph,500.0,0.001754,0.000694,Single Segment,Start|End,2023-09-20,2023-10-10,20 days
726,726,577755,place-gilmn | place-esomr,Active Restriction,Track,WB,Green Line,Green Line Trunk,GL Kenmore-College Ave WB,Between Stations,03 mph,400.0,0.001403,0.000555,Single Segment,Start|End,2023-09-20,2023-10-10,20 days
727,727,577756,place-gilmn | place-esomr,Active Restriction,Track,WB,Green Line,Green Line Trunk,GL Kenmore-College Ave WB,Between Stations,03 mph,400.0,0.001403,0.000555,Single Segment,Start|End,2023-09-20,2023-10-10,20 days


In [236]:
# Define the marker and cluster dictionaries, along with the feature group for the map.
speed_restriction_markers = {}
cluster_dict = {}
cluster_feature_group = fol.FeatureGroup('restriction_clusters', overlay=True)

# Go through each unique stop id in the speed restriction data and create a cluster for the location.
for stop_id in set(sr_df.Loc_GTFS_Stop_ID.array):
    cluster = MarkerCluster(options={
        # 'singleMarkerMode' : True,
        'showCoverageOnHover' : False,
        'maxClusterRadius' : 160,

        }).add_to(cluster_feature_group)
    cluster_dict[stop_id] = cluster

# Run through each speed restriction, extract the information about the restriction and add it to a marker that represents it on the map.
for i in range(len(sr_df)):
    # Attributes for tags
    speed = sr_df.iloc[i]['Restriction_Speed_MPH']
    distance = sr_df.iloc[i]['Restriction_Distance_Feet']
    pct = sr_df.iloc[i]['Line_Restricted_Track_Pct']
    length = sr_df.iloc[i]['Restriction_Length_Days']
    start_date = sr_df.iloc[i]['start_date']
    end_date = sr_df.iloc[i]['end_date']
    loc_type = sr_df.iloc[i]['Location_Type']
    line = str(sr_df.iloc[i]['Line'])
    stop_id = sr_df.iloc[i]['Loc_GTFS_Stop_ID']

    # Process the stop_id to extract the name of the station(s)
    if ' | ' in stop_id:
        stations = [stop_name_from_stop_id(id) for id in stop_id.split(' | ')]
        name = f'Between {stations[0]} and {stations[1]}'
    else: name = stop_name_from_stop_id(stop_id)

    # Determine Location for marker
    location_point = get_location_from_stop_id(stop_id)
    location = location_point.coords.xy[0][0], location_point.coords.xy[1][0]
    converted_long, converted_lat = to_lat_long(location[0], location[1])

    # Create Marker
    marker = fol.Marker(
        location=[converted_lat, converted_long],
        popup = fol.Popup(f'Restriction Speed: {speed}\nRestriction Distance: {distance} ft\nPercent of Line Restricted: {pct : 2.2%}\nRestriction Length: {length} ({start_date} to {end_date})'),
        tags=[speed, loc_type],
        tooltip=name,
        icon=fol.Icon(
            color=line.split()[0].lower(),
            icon='triangle-exclamation',
            prefix='fa'
            )
    )

    # Add the marker to the cluster defined in the cluster dictionary
    cluster = cluster_dict[stop_id]
    marker.add_to(base_map)

In [237]:
# Extract all unique speeds for all restrictions to create the filter for the map.
speeds = list(set(sr_df.Restriction_Speed_MPH.values))

# Extract all location types for the filter on the map
loc_types = list(set(sr_df.Location_Type.values))

# Define the filter values and sort them
filters = speeds + loc_types
filters.sort()

In [238]:
# Format the cluster feature group, filter, and fullscreen buttons
cluster_feature_group.add_to(base_map)
fullscreen = plug.Fullscreen(position='bottomright').add_to(base_map)
filter = plug.TagFilterButton(filters, clear_text='Clear Filters').add_to(base_map)

In [239]:
# Save the map
base_map.save('speed_restrictions_interactive_v4.html')